## Resumo aula 4 de Machine Learning ##

no modelo de CLASSIFICAÇÃO KNN, 
k=1 temos alta variância (overfitting - pense nos outliers!).

k=N temos alto viés (underfittinf - não aprendemos nada, só predizemos diretamente a classe majoritária na base de treino).

In [114]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_validate, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, classification_report, make_scorer, precision_score, recall_score, accuracy_score, f1_score



In [115]:
bcancer_ds=pd.read_csv('data.csv')
bcancer_ds.info()
bcancer_ds = bcancer_ds.drop("Unnamed: 32", axis=1)
bcancer_raw = bcancer_ds.copy()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

In [116]:
y = bcancer_ds['diagnosis']
X = bcancer_ds.drop(columns='diagnosis', axis=1)
x_train, x_test, y_train, y_test = train_test_split(X,y)


In [117]:
scaler=StandardScaler()
bcancer_scaler = scaler.fit(x_train)
x_train_ss = bcancer_scaler.transform(x_train)
x_test_ss = bcancer_scaler.transform(x_test)

In [118]:
knn = KNeighborsClassifier()
kf5 = KFold(n_splits=5, shuffle=True)
# knn.fit(x_train_ss,y_train)

# y_predict = knn.predict(x_test_ss)
# scoring =('accuracy','precision','f1','recall')
# cm = confusion_matrix(y_test, y_predict)
# dl=["B","M"]
# disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=dl)
# disp.plot()
scoring = {
    'precision': make_scorer(precision_score, pos_label='M'),
    'recall': make_scorer(recall_score, pos_label='M'),
    'f1': make_scorer(f1_score, pos_label='M'),
    'accuracy': make_scorer(accuracy_score)
}

In [119]:
cv = cross_validate(estimator=knn, X=x_train_ss, y=y_train,cv=kf5 ,scoring=scoring,return_train_score=True)

pd.DataFrame(cv)

,fit_time,score_time,test_precision,train_precision,test_recall,train_recall,test_f1,train_f1,test_accuracy,train_accuracy
0,0.003002,0.010999,0.960000,0.991803,0.960000,0.909774,0.960000,0.949020,0.976744,0.961765
1,0.002001,0.009001,1.000000,0.983607,0.896552,0.930233,0.945455,0.956175,0.964706,0.967742
2,0.001000,0.009002,0.939394,0.991304,0.968750,0.904762,0.953846,0.946058,0.964706,0.961877
3,0.000999,0.009998,1.000000,0.990909,0.871795,0.915966,0.931507,0.951965,0.941176,0.967742
4,0.002000,0.009999,1.000000,0.991736,0.757576,0.960000,0.862069,0.975610,0.905882,0.982405
